In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [3]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [4]:
config = initializeConfig()
obs = getObs(400)

In [5]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30075/1048845639.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[i].append(obs[i].mean()['Stability Margin (cal)'])


,0,1,2,3,4,5,6,7,8,9,10
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,73446.170000,0.853824,150.9995


In [6]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [7]:
import torch
from torch.utils import data
from torch import nn

In [8]:
def synthetic_data(m, c, num_examples):
 
    """Generate y = mX + bias(c) + noise"""

    X = torch.normal(0, 1, (num_examples, len(m)))

    y = torch.matmul(X, m) + c

    y += torch.normal(0, 0.01, y.shape)
 
    return X, y.reshape((-1, 1))

In [9]:
synthetic_data(torch.Tensor([4,2]),2,10)

(tensor([[-0.7479, -0.0492],
         [ 1.3770,  0.4481],
         [-0.4976, -0.6702],
         [-0.3553,  0.2949],
         [ 1.2188,  0.2726],
         [-0.0802, -1.7648],
         [-1.0519,  0.0761],
         [ 1.5940,  0.0251],
         [ 1.8004,  0.3922],
         [ 0.6819,  0.1250]]),
 tensor([[-1.0883],
         [ 8.4088],
         [-1.3344],
         [ 1.1678],
         [ 7.4460],
         [-1.8586],
         [-2.0622],
         [ 8.4260],
         [ 9.9924],
         [ 4.9801]]))

In [10]:
filtered_df = df[df['Time'] > 90]
filtered_df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


In [11]:
poly = PolynomialFeatures(2)
raw_X = filtered_df.iloc[:,:8]
raw_Y = filtered_df.iloc[:,-3]
batch_size = 10

In [12]:
train_X = torch.from_numpy(poly.fit_transform(raw_X[:100])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:100]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[100:])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[100:]])).double()

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30075/2555709260.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:100]])).double()
/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30075/2555709260.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[100:]])).double()


In [13]:
def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [14]:
data_iter = load_arrays((train_X, train_Y), 10)

In [15]:
data_iter

In [92]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 10)
        self.activation2 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
#         x = self.linear3(x)
#         x = self.activation3(x)
        x = self.linear5(x)
        
        return 1000000*x

In [93]:
net = Net()
net.double()
net(train_X)

tensor([[ 3971860.6167],
        [ 5483843.2353],
        [ 4374941.6994],
        [ 2048962.6525],
        [ 1866850.9080],
        [ 5404353.2239],
        [ 5906977.9858],
        [ 3148442.2971],
        [ 2623731.5015],
        [ 4120257.2304],
        [ 2783717.0876],
        [ 3630098.8636],
        [ 6276378.8541],
        [ 4105875.9022],
        [ 2794997.9195],
        [ 3018899.3693],
        [ 3762110.6715],
        [ 2559879.5416],
        [ 4179624.3094],
        [ 3231408.4435],
        [ 7677704.5710],
        [ 5898834.8394],
        [ 3810764.3728],
        [ 7470913.1393],
        [ 2531440.1301],
        [ 5231520.5385],
        [ 3801685.5969],
        [ 1679913.0595],
        [ 3199971.5990],
        [ 5167602.8386],
        [ 2639103.3384],
        [ 4595180.6563],
        [  276122.9633],
        [ 4796036.8236],
        [ 3594873.7563],
        [ 3919238.9916],
        [ 2422408.6004],
        [ 1376784.6688],
        [ 4795676.4192],
        [ 3103106.2986],


In [94]:
loss = nn.MSELoss()

In [95]:
trainer = torch.optim.SGD(net.parameters(), lr= 0.0000000000000001) # 0.000001, 0.0000000000000001 00000000000000001
net(train_X)

tensor([[ 3971860.6167],
        [ 5483843.2353],
        [ 4374941.6994],
        [ 2048962.6525],
        [ 1866850.9080],
        [ 5404353.2239],
        [ 5906977.9858],
        [ 3148442.2971],
        [ 2623731.5015],
        [ 4120257.2304],
        [ 2783717.0876],
        [ 3630098.8636],
        [ 6276378.8541],
        [ 4105875.9022],
        [ 2794997.9195],
        [ 3018899.3693],
        [ 3762110.6715],
        [ 2559879.5416],
        [ 4179624.3094],
        [ 3231408.4435],
        [ 7677704.5710],
        [ 5898834.8394],
        [ 3810764.3728],
        [ 7470913.1393],
        [ 2531440.1301],
        [ 5231520.5385],
        [ 3801685.5969],
        [ 1679913.0595],
        [ 3199971.5990],
        [ 5167602.8386],
        [ 2639103.3384],
        [ 4595180.6563],
        [  276122.9633],
        [ 4796036.8236],
        [ 3594873.7563],
        [ 3919238.9916],
        [ 2422408.6004],
        [ 1376784.6688],
        [ 4795676.4192],
        [ 3103106.2986],


In [96]:
num_epochs = 1000
 
for epoch in range(num_epochs):
 
    for X, y in data_iter:

        l = loss(net(X) , 1000000*y)

        trainer.zero_grad() #sets gradients to zero

        l.backward() # back propagation

        trainer.step() # parameter update

    l = loss(net(train_X), train_Y)

    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 763602230350.533691
epoch 2, loss 501881290845.669556
epoch 3, loss 400525718163.339600
epoch 4, loss 378148995219.604004
epoch 5, loss 331894648738.352600
epoch 6, loss 352311633981.403687
epoch 7, loss 357851111568.210815
epoch 8, loss 335792434426.832764
epoch 9, loss 360939662857.585815
epoch 10, loss 337168431383.426575
epoch 11, loss 344598391380.853638
epoch 12, loss 313406586302.296326
epoch 13, loss 327293125627.011414
epoch 14, loss 304968227783.077698
epoch 15, loss 337031350101.028015
epoch 16, loss 334109181671.228149
epoch 17, loss 354958301988.043457
epoch 18, loss 341123063676.025391
epoch 19, loss 339720930973.802185
epoch 20, loss 375305816414.932678
epoch 21, loss 340947851694.933655
epoch 22, loss 367215283090.242737
epoch 23, loss 343815792644.705566
epoch 24, loss 362151059378.650940
epoch 25, loss 380137984851.180115
epoch 26, loss 370916222440.037720
epoch 27, loss 380112648851.522583
epoch 28, loss 373299563269.500122
epoch 29, loss 393293563439.6

epoch 254, loss 420126804533.162720
epoch 255, loss 403699667405.233459
epoch 256, loss 409691235801.620667
epoch 257, loss 431674680977.533142
epoch 258, loss 413654272540.111023
epoch 259, loss 411347009980.941284
epoch 260, loss 405631311189.342896
epoch 261, loss 410883447436.745911
epoch 262, loss 396780886821.165833
epoch 263, loss 401581817887.231689
epoch 264, loss 440030721499.619080
epoch 265, loss 425414040320.830933
epoch 266, loss 400080964384.168762
epoch 267, loss 387023606472.943542
epoch 268, loss 395389949138.027893
epoch 269, loss 400648866581.515747
epoch 270, loss 404597223783.500610
epoch 271, loss 412048149579.930847
epoch 272, loss 413927587639.357117
epoch 273, loss 416415221238.856018
epoch 274, loss 426998763343.945496
epoch 275, loss 415883853004.812134
epoch 276, loss 411559551108.914429
epoch 277, loss 430690660090.572327
epoch 278, loss 389678488014.542603
epoch 279, loss 406631478228.773682
epoch 280, loss 400555216990.345703
epoch 281, loss 405135559542

epoch 495, loss 421981727719.042175
epoch 496, loss 427684614199.044189
epoch 497, loss 427442857369.383362
epoch 498, loss 417676831876.730652
epoch 499, loss 424053233958.949402
epoch 500, loss 430310853193.637512
epoch 501, loss 411598869005.738098
epoch 502, loss 406146802193.663086
epoch 503, loss 400996669246.723083
epoch 504, loss 407667890191.464050
epoch 505, loss 423860717601.344543
epoch 506, loss 418531499957.200623
epoch 507, loss 434638888117.408386
epoch 508, loss 431863227492.372681
epoch 509, loss 420899029027.615967
epoch 510, loss 413977824076.635925
epoch 511, loss 399669566804.402710
epoch 512, loss 393508758358.025391
epoch 513, loss 399124326524.765625
epoch 514, loss 411635185789.233521
epoch 515, loss 437944447861.999268
epoch 516, loss 425644135593.355957
epoch 517, loss 418333054612.065186
epoch 518, loss 413154089819.883179
epoch 519, loss 405827411859.404663
epoch 520, loss 411561671242.105713
epoch 521, loss 411096697323.520081
epoch 522, loss 431629767890

epoch 745, loss 428190868907.493225
epoch 746, loss 422284746981.795959
epoch 747, loss 434091611392.606201
epoch 748, loss 421065356980.133301
epoch 749, loss 424212214653.077515
epoch 750, loss 422500251307.505371
epoch 751, loss 408622672129.662048
epoch 752, loss 451882590286.714722
epoch 753, loss 424299280861.727234
epoch 754, loss 426386576170.837219
epoch 755, loss 450071535920.413452
epoch 756, loss 396592719506.055176
epoch 757, loss 419703348842.099915
epoch 758, loss 459145198151.860718
epoch 759, loss 415138612024.964844
epoch 760, loss 446265442388.040955
epoch 761, loss 392473341111.713013
epoch 762, loss 448463264998.771362
epoch 763, loss 398471651725.953308
epoch 764, loss 402785028053.565247
epoch 765, loss 440013952756.122986
epoch 766, loss 419007802254.282837
epoch 767, loss 442086117743.271729
epoch 768, loss 442248173422.249390
epoch 769, loss 456698247108.959045
epoch 770, loss 418694655905.669189
epoch 771, loss 423183298049.623291
epoch 772, loss 440593777097

epoch 986, loss 432393388368.088318
epoch 987, loss 421175549608.933594
epoch 988, loss 431411270345.369873
epoch 989, loss 416863144632.740112
epoch 990, loss 425365126232.818420
epoch 991, loss 435687355919.934387
epoch 992, loss 441708379777.803101
epoch 993, loss 405332901423.714050
epoch 994, loss 455268574070.827087
epoch 995, loss 410590724889.120117
epoch 996, loss 425960590831.725891
epoch 997, loss 408009289936.707336
epoch 998, loss 423498809199.630066
epoch 999, loss 441209718115.673828
epoch 1000, loss 406888077436.931458


In [97]:
np.sqrt(l.detach().numpy()/100)

63787.77919295603

In [22]:
np.sqrt((loss(net(test_X), test_Y)/100).detach().numpy())

4.2326902124505875e+42

In [23]:
# NN: weird landscape. Need to start with less stepsize (e.g. 0.000001) and then grow to 0.01

In [24]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [39]:
classif = Classifier()
classif.double()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [47]:
poly = PolynomialFeatures(2)
raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10
idxs = np.array(list(range(300, 400)) + list(range(200)))

train_X = torch.from_numpy(poly.fit_transform(raw_X.iloc[idxs])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y.iloc[idxs]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[200:300])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[200:300]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [48]:
classif(train_X)

tensor([[1.1788e-06],
        [6.6432e-19],
        [4.5428e-22],
        [9.9993e-01],
        [2.5500e-14],
        [1.0000e+00],
        [2.1470e-27],
        [9.9954e-01],
        [1.0000e+00],
        [9.9997e-01],
        [1.0000e+00],
        [1.6892e-17],
        [9.9998e-01],
        [9.9998e-01],
        [2.0292e-04],
        [9.9999e-01],
        [1.1266e-23],
        [9.9373e-01],
        [9.9999e-01],
        [1.0000e+00],
        [1.7501e-22],
        [1.0000e+00],
        [1.0000e+00],
        [1.2754e-21],
        [6.2191e-11],
        [1.0000e+00],
        [1.0000e+00],
        [3.0289e-04],
        [9.9998e-01],
        [2.3709e-14],
        [3.9235e-05],
        [7.0549e-18],
        [3.2280e-05],
        [9.9998e-01],
        [1.2813e-11],
        [6.4680e-08],
        [1.0000e+00],
        [2.0637e-04],
        [1.0000e+00],
        [5.2933e-13],
        [1.0316e-18],
        [9.9997e-01],
        [9.9997e-01],
        [1.0000e+00],
        [1.0000e+00],
        [9

In [49]:
loss = nn.BCELoss()

In [50]:
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
for epoch in range(num_epochs):
 
    for X, y in data_iter:
        X.requires_grad = True
        l = loss(classif(X) ,y)

        trainer.zero_grad() #sets gradients to zero

        l.backward() # back propagation

        trainer.step() # parameter update
        
    l = loss(classif(train_X), train_Y)
    if epoch % 50 == 0:
        print(f'epoch {epoch + 1}, loss {l:f}')
        #print(X.grad)

epoch 1, loss 0.251323
epoch 51, loss 0.007805
epoch 101, loss 0.002101
epoch 151, loss 0.001046
epoch 201, loss 0.000606


In [51]:
loss(classif(test_X), test_Y)

tensor(0.0331, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)

In [52]:
sum((classif(test_X) >= 0.5) == test_Y)/len(test_Y)

tensor([0.9800])

In [ ]:
def coordinateAscentNN(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2.5) and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier.predict_proba(poly.fit_transform([y]))[0,1]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [ ]:
def transform(x):
    L = [torch.Tensor([1]), x]
    for i in range(len(x)):
        for j in range(i, len(x)):
            L.append(torch.Tensor([x[i]*x[j]]))
    return torch.cat(L).double()

In [ ]:
transform(x)

In [ ]:
x = torch.Tensor([3,10,6,7,9,2.1,5,2])
# x.requires_grad = True
z = transform(x)
z.requires_grad = True
out = net(z)
#torch.autograd.grad(outputs = out, inputs = x, retain_graph = True)
x.grad

In [ ]:
out.backward()

In [ ]:
z.grad.data

In [ ]:
out[0]

In [101]:
idxs = [0,1,2,3,4,5,6,7,-1,-2,-3]

In [102]:
df.iloc[:,idxs].mean()

Schord           5.867377
Sspan            5.828527
Ssweep           5.963500
Stip             6.026743
Bchord           6.025729
Bspan            5.898697
Bsweep           6.174065
Btip             5.949068
Time            62.544382
Stability        0.634584
Altitude     26332.898899
dtype: float64